# Q1

H0 : False H1 : True일 때
FP: 실제 False인 정답을 True라고 예측 (오답) -> Type 1 error
FN: 실제 True인 정답을 False라고 예측 (오답) -> Type 2 error
Type 1 error과 Type 2 error는 trade off 관계이다.
따라서 FP와 FN 도 Trade off 관계에 있다

# Q2-1

Accuracy(정확도: 전체 예측 중 옳게 말한 결과의 비율) = (TP+TN) / (TP+TN+FP+FN)
Precision(정밀도: True라고 예측한 것 중 실제로 True의 비율) = TP / (TP+FP)
Recall(재현율: 실제 True 중 True라고 예측한 것의 비율) = TP / (TP+FN)

# Q2-2

In [ ]:
의사가 병의 진단을 내릴 때,
H0: 병이 아니다, H1: 병이다
이 경우, 병에 걸린 사람을 멀쩡하다고 진단하는 것을 막는 것이, 병에 걸리지 않은 사람이 위험하다고 진단하는 것을 막는 것보다 훨씬 중요하다.
이 경우 threshold를 늘리는 것은 전자의 경우를 더 허용하는 것이므로 바람직하지 않다

# Q3

In [12]:
# import
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
# For reproducibility 
torch.manual_seed(1)

In [5]:
# Hyper Parameters  
input_size = 784 # input size is 784
num_classes = 10 # 10 digits are present in this and hence, we can have 10 different outputs
num_epochs = 5 
batch_size = 100 # will train in small batches of 100 images
learning_rate = 0.001

In [6]:
# MNIST Dataset (Images and Labels)  # 데이터셋 불러오기
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 

test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, # from torch.utils.data import Dataset 
                                           batch_size = batch_size, # 각 minibatch의 크기, 통상적으로 2의 제곱수로 설정
                                           shuffle = True)          # Epoch마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꾼다
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)


In [7]:
class LogisticRegression(nn.Module): # 무슨 모델 이용할 건지 정하기
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) # 선형 모델
  
    def forward(self, x): # define the forward pass
        out = self.linear(x) 
        return out 

In [8]:
model = LogisticRegression(input_size, num_classes) 

In [9]:
criterion = nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # optimizer

In [21]:
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() # Reset all gradients to 0 (0으로 초기화)
        outputs = model(images) # Make a forward pass (첫 예측)
        loss = criterion(outputs, labels) # Calculate the loss (loss 어떻게 계산할 건지)
        loss.backward() # Perform backpropagation (계산)
        optimizer.step() # Update all weights
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data)) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2144
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1163
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0258
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9763
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.9312
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7738
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7256
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6739
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.6204
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5783
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5544
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4273
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.4927
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.3208
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3455
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.4228
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.3079
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.3068
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2157
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1718
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [22]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28))
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) # ㅡmax 의 label 로 predict
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
    
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  82 %


# Q4-1

 모르겠다 왜 알아야 하지.. SGD를 사용하지 못하는 경우가 있나?

# Q4-2

In [18]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1) 
    
    def forward(self, x):
        return self.linear(x)
    

### SGD

In [19]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.066, b: -0.150 Cost: 0.003239
Epoch  200/1000 W: 1.052, b: -0.118 Cost: 0.002002
Epoch  300/1000 W: 1.041, b: -0.093 Cost: 0.001237
Epoch  400/1000 W: 1.032, b: -0.073 Cost: 0.000764
Epoch  500/1000 W: 1.025, b: -0.057 Cost: 0.000472
Epoch  600/1000 W: 1.020, b: -0.045 Cost: 0.000292
Epoch  700/1000 W: 1.016, b: -0.035 Cost: 0.000180
Epoch  800/1000 W: 1.012, b: -0.028 Cost: 0.000111
Epoch  900/1000 W: 1.010, b: -0.022 Cost: 0.000069
Epoch 1000/1000 W: 1.008, b: -0.017 Cost: 0.000043


### adam

In [20]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.525, b: -0.431 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.032 Cost: 0.000333
Epoch  200/1000 W: 0.990, b: 0.022 Cost: 0.000072
Epoch  300/1000 W: 0.993, b: 0.017 Cost: 0.000040
Epoch  400/1000 W: 0.995, b: 0.012 Cost: 0.000020
Epoch  500/1000 W: 0.997, b: 0.008 Cost: 0.000008
Epoch  600/1000 W: 0.998, b: 0.005 Cost: 0.000003
Epoch  700/1000 W: 0.999, b: 0.003 Cost: 0.000001
Epoch  800/1000 W: 0.999, b: 0.002 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


### rmsprop

In [21]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01) 
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.615, b: -0.341 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.019 Cost: 0.000052
Epoch  200/1000 W: 0.996, b: 0.008 Cost: 0.000010
Epoch  300/1000 W: 0.999, b: 0.002 Cost: 0.000001
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 0.984, b: -0.016 Cost: 0.002620
Epoch  800/1000 W: 0.999, b: -0.001 Cost: 0.000006
Epoch  900/1000 W: 0.998, b: -0.002 Cost: 0.000030
Epoch 1000/1000 W: 0.996, b: -0.004 Cost: 0.000109


### SGD with momentum

In [23]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
model = LinearRegressionModel() 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 교차 검증할 거면 0.5, 0.9, 0.95, 0.99 순서로 증가시켜 검증
nb_epochs = 1000 

for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = model(x_train)        

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)        

    # cost로 H(x) 개선    
    optimizer.zero_grad() # 미분값이     
    cost.backward()    
    optimizer.step()        

    # 100번마다 로그 출력    
    if epoch % 100 == 0:        
        params = list(model.parameters())        
        W = params[0].item()        
        b = params[1].item()        
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()        
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.007, b: -0.008 Cost: 0.000053
Epoch  200/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: -0.000 Cost: 0.000000
